# Scikit-learn from CSV

This notebook reads the CSV data written out by the Dataflow program of [1_explore.ipynb](./1_explore.ipynb) and trains a scikit-learn model on Cloud ML Engine.


In [ ]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [ ]:
%%bash
pip install cloudml-hypertune

## Train sklearn model locally

In [ ]:
%%bash
OUTDIR=skl_trained
DATADIR=${PWD}/preproc/csv
rm -rf $OUTDIR
gcloud ml-engine local train \
    --module-name=trainer.train_skl --package-path=${PWD}/ltgpred/trainer \
    -- \
    --job-dir=$OUTDIR --train_data=${DATADIR}/train* --eval_data=${DATADIR}/eval*

## Training sklearn model on CMLE

In [ ]:
%writefile largemachine.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_l

In [8]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/skl_trained
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/csv
JOBNAME=ltgpred_skl_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud alpha ml-engine jobs submit training $JOBNAME \
    --module-name=ltgpred.trainer.train_skl --package-path=${PWD}/ltgpred --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=custom --config=largemachine.yaml \
    --python-version=3.5 --runtime-version=1.8 \
    -- \
    --train_data=${DATADIR}/train-001* --eval_data=${DATADIR}/eval-0000*

jobId: ltgpred_skl_181222_012654
state: QUEUED


Removing gs://cloud-training-demos-ml/lightning/skl_trained/packages/0b79740ddb029ba0ac53dea461ad8cf1253b1209d39691c1340af0416109d24b/ltgpred-0.0.1.tar.gz#1545441204052529...
/ [1/1 objects] 100% Done                                                       
Operation completed over 1 objects.                                              
Job [ltgpred_skl_181222_012654] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe ltgpred_skl_181222_012654

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs ltgpred_skl_181222_012654


When I ran it, training finished with a RMSE=0.34.
This can serve as a benchmark.

Note, however, that I trained and evaluated on a subset of the data, since even the "largemachine" doesn't have the memory needed to hold entire dataset

Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License